In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
print("TensorFlow version:", tf.__version__)
print("Pandas version:", pd.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.10.1
Pandas version: 2.2.3
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
df_Regions = pd.read_csv(r'C:\Users\ROG Strix\Desktop\Predict\NCAA Bracket\Team Region Groups.csv')

In [3]:
df = pd.read_csv(r'C:\Users\ROG Strix\Desktop\Predict\NCAA Bracket\games_2022.csv')

In [4]:
df = df.drop(columns=['home_away', 'tz_dif_H_E','notD1_incomplete'], axis=1)

In [5]:
df["pts_Differential"] = df["team_score"] - df["opponent_team_score"]

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
#win=1, lose = 0
df["Won"] = df.apply(lambda row: 1 if row["team_score"] > row["opponent_team_score"] 
                    else 0, axis=1)

In [8]:
#replaced nulls with 0s
df["OT_length_min_tot"].fillna(0, inplace=True)

In [9]:
df = df_Regions.merge(df, how="right", on="team")
#Add regions to the df

In [10]:
df["region"].fillna('East', inplace=True)

In [11]:
#group teams by region to make sure i didnt fuck up w the last step
teams_by_region = df.groupby("region")["team"].unique()

In [12]:
# Replacing Nulls values with 0 
# I do not know what to do with these values just yet so i just =0
df[["attendance", "rest_days","prev_game_dist","travel_dist","largest_lead"]] = df[["attendance", "rest_days","prev_game_dist","travel_dist","largest_lead"]].fillna(0)

In [13]:
df["cumulative_games_played"] = df.groupby("team").cumcount()
df["cumulative_Won"] = (df.groupby("team", group_keys=False)["Won"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["Winning_Percentage"] = (df["cumulative_Won"] / df["cumulative_games_played"]).fillna(0)

df["cumulative_BLK"] = (df.groupby("team", group_keys=False)["BLK"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_STL"] = (df.groupby("team", group_keys=False)["STL"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_TOV"] = (df.groupby("team", group_keys=False)["TOV"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_F_personal"] = (df.groupby("team", group_keys=False)["F_personal"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_AST"] = (df.groupby("team", group_keys=False)["AST"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_OREB"] = (df.groupby("team", group_keys=False)["OREB"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_DREB"] = (df.groupby("team", group_keys=False)["DREB"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_DREB"] = (df.groupby("team", group_keys=False)["DREB"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
#Reset the index again if needed
df = df.reset_index(drop=True)

In [14]:
#make copy of df that will represent the opponent stats.
df_opp = df.copy()

# all columns add an "opp_" prefix.
df_opp = df_opp.rename(columns=lambda x: x if x == "game_id" else "opp_team" if x == "team" else "opp_" + x)

# merge the original df with df_opp on the game_id.
df = pd.merge(df, df_opp, on="game_id", how="inner")

# remove  rows where the team is paired with itself.
df = df[df["team"] != df["opp_team"]]

#  Reset the index
df = df.reset_index(drop=True)

In [15]:
# Field Goal Percentage
df["fg_2_per"] = df.apply(lambda row: (row["FGM_2"] / row["FGA_2"]) if row["FGA_2"] > 0 else 0, axis=1)
df["fg_3_per"] = df.apply(lambda row: (row["FGM_3"] / row["FGA_3"]) if row["FGA_3"] > 0 else 0, axis=1)
df["ft_per"] = df.apply(lambda row: (row["FTM"] / row["FTA"]) if row["FTA"] > 0 else 0, axis=1)
#effective Field Goal Percentage
df["efg_per"] = df.apply(lambda row: (row["FGM_2"] + 1.5 * row["FGM_3"]) / (row["FGA_2"] + row["FGA_3"]) if (row["FGA_2"] + row["FGA_3"]) > 0 else 0, axis=1)

cumulative_cols = [
    "Won", "FGM_2", "FGM_3", "FGA_2", "FGA_3", "FTA", "FTM", "efg_per", "fg_3_per", "fg_2_per", 
    "largest_lead","team_score","opponent_team_score"]

# Apply fix to all cumulative features
for col in cumulative_cols:
    df[f"cumulative_{col}"] = (df.groupby("team", group_keys=False)[col]
                                .apply(lambda x: x.shift(1).expanding().sum())
                                .fillna(0))


# Calculate cumulative losses (using only past data)
df["cumulative_wins"] = df["cumulative_Won"]  # Won is already shifted
df["cumulative_losses"] = df["cumulative_games_played"] - df["cumulative_wins"]

# Calculate cumulative wins and games played for each tea
df["win_loss_ratio"] = df["cumulative_wins"] / df["cumulative_losses"].replace(0, np.nan)  # Replace 0 with NaN to avoid div by zero
df["win_loss_ratio"] = df["win_loss_ratio"].fillna(0)  # replace NaN cases with 0

df["win_per"] = df["cumulative_wins"] / df["cumulative_games_played"]
df["win_per"] = df["win_per"].fillna(0)  # replace NaN cases with 0


#3pt PAR(3pt attepmt rate)
df["3PAR"] = df.apply(lambda row: (row["FGA_3"]) / (row["FGA_2"] + row["FGA_3"]) if (row["FGA_2"] + row["FGA_3"]) > 0 else 0, axis=1)

#FTR(free throw rate)
df["FTR"] = df.apply(lambda row: (row["FTA"]) / (row["FGA_2"] + row["FGA_3"]) if (row["FGA_2"] + row["FGA_3"]) > 0 else 0, axis=1)

#Assist Turnover Ratio
df["ASTTO_ratio"] = df.apply(lambda row: (row["AST"] /(row["TOV"] + row["TOV_team"])) if (row["TOV"] + row["TOV_team"]) > 0 else 0, axis=1)

#True shooting percentage
df["TS_per"] = df.apply(lambda row: (row["team_score"] )/(2*((row["FGA_2"] + row["FGA_3"]) + 0.44 * row["FTA"])) if (2*((row["FGA_2"] + row["FGA_3"]) + 0.44 * row["FTA"])) > 0 else 0, axis=1)

In [16]:
redundant_columns = [
    'opp_game_date', 'opp_team_score', 'opp_opponent_team_score', 'opp_largest_lead', 
    'opp_OT_length_min_tot', 'opp_attendance',
    'opp_pts_Differential', 'opp_Won']
# Drop the redundant columns from the dataframe
df = df.drop(columns=redundant_columns)

In [17]:
# Create df_train and ensure no nulls in essential columns
df_train = pd.DataFrame()
df_train["team"] = df["team"]

df["game_id"] = df["game_id"].map(lambda x: x.lstrip("game_"))
df_train["game_id"]=df["game_id"]
train = df_train.pop("game_id")
df_train.insert(0, "game_id", train) #Move "game_id" to front

df_train["Region"] = df["region"]
train2 = df_train.pop("Region")
df_train.insert(2, "Region", train2) #Move "game_id" to front

df_train["game_date"] = df["game_date"]
train3 = df_train.pop("game_date")
df_train.insert(1, "game_date", train3) #Move "game_id" to front

# Avoid division by zero and handle NaNs
df_train["WinRatioPerGame"] = df["Winning_Percentage"]
df_train["PtsPerGame"] = (df["cumulative_team_score"] / df["cumulative_games_played"]).fillna(0)  # Points per game
df_train["PointRatio"] = (df_train["PtsPerGame"] / (df["cumulative_opponent_team_score"] / df["cumulative_games_played"])).fillna(0)  # Points versus opponent point ratio

df_train["FGPerGame"] = ((df["cumulative_FGM_2"] + df["cumulative_FGM_3"]) / df["cumulative_games_played"]).fillna(0)  # Shots made per game
df_train["FGAPerGame"] = ((df["cumulative_FGA_2"] + df["cumulative_FGA_3"]) / df["cumulative_games_played"]).fillna(0)  # Misses per game
df_train["FGRatio"] = (df_train["FGPerGame"] / df_train["FGAPerGame"]).fillna(0)  # Field goal percentage per game

df_train["FG_3_PerGame"] = (df["cumulative_FGM_3"] / df["cumulative_games_played"]).fillna(0)  # 3-pointer made per game
df_train["FGA_3_PerGame"] = (df["cumulative_FGA_3"] / df["cumulative_games_played"]).fillna(0)  # 3-pointer attempts per game
df_train["FG_3_Ratio"] = (df_train["FG_3_PerGame"] / df_train["FGA_3_PerGame"]).fillna(0)  # 3-pointer field goal percentage

df_train["FTMPerGame"] = (df["cumulative_FTM"] / df["cumulative_games_played"]).fillna(0)  # Free throw made per game
df_train["FTAPerGame"] = (df["cumulative_FTA"] / df["cumulative_games_played"]).fillna(0)  # Free throw attempt per game
df_train["FTRatio"] = (df_train["FTMPerGame"] / df_train["FTAPerGame"]).fillna(0)  # Free throw goal percentage

df_train["STLPerGame"] = (df["cumulative_STL"] / df["cumulative_games_played"]).fillna(0)  # Steals per game
df_train["TOVPerGame"] = (df["cumulative_TOV"] / df["cumulative_games_played"]).fillna(0)  # Turnovers per game
df_train["BLKPerGame"] = (df["cumulative_BLK"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game
df_train["FoulPerGame"] = (df["cumulative_F_personal"] / df["cumulative_games_played"]).fillna(0)  # Fouls per game

df_train["FG_2_PPerGame"] = (df["cumulative_fg_2_per"] / df["cumulative_games_played"]).fillna(0)  # Field goal percentage per game (2pt)
df_train["FG_3_PPerGame"] = (df["cumulative_fg_3_per"] / df["cumulative_games_played"]).fillna(0)  # Field goal percentage per game (3pt)
df_train["eFGPerGame"] = (df["cumulative_efg_per"] / df["cumulative_games_played"]).fillna(0)  # Effective Field Goal Percentage per game (3pt) 

df_train["OREBinPossesions"] = (df["cumulative_OREB"] / (df["cumulative_OREB"] + df["opp_cumulative_DREB"])).fillna(0)  # Offensive rebound: Rebounds
df_train["DREBinPossesions"] = (df["cumulative_DREB"] / (df["cumulative_OREB"] + df["opp_cumulative_DREB"])).fillna(0)  # Defensive rebound: Rebounds

df_train["largestLeadPerGame"] = (df["cumulative_largest_lead"] / df["cumulative_games_played"]).fillna(0)  # Largest lead per game

df_train["OREBRatio"] = (df["cumulative_OREB"] / df["cumulative_games_played"]).fillna(0)  # Offensive rebound per game
df_train["DREBRatio"] = (df["cumulative_DREB"] / df["cumulative_games_played"]).fillna(0)  # Defensive rebound per game

df_train["BLKPerGame"] = (df["cumulative_BLK"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game
df_train["FoulPerGame"] = (df["cumulative_F_personal"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game
df_train["TOVPerGame"] = (df["cumulative_TOV"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game
df_train["STLPerGame"] = (df["cumulative_STL"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game
df_train["ASTPerGame"] = (df["cumulative_AST"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game

In [18]:
# Step 1: Shift 'Won' column by 1 for each team (ensuring only past data is used)
df["Won_Shifted"] = df.groupby("team")["Won"].shift(1).fillna(0)

# Step 2: Only use opponent's winning percentage **if the team has played at least one game**
df["PowerPoints"] = df.apply(lambda row: row["Winning_Percentage"] + 
                             (row["opp_Winning_Percentage"] if row["cumulative_games_played"] > 0  
                              else 0), axis=1)

# Step 3: Compute Cumulative Power Rank **only for teams that have played at least one game**
df["PowerRank"] = df.groupby("team")["PowerPoints"].cumsum().where(df["cumulative_games_played"] > 0, 0)

# Ensure Power Rank is in df_train
df_train["PowerRank"] = df["PowerRank"]

In [19]:
df_train.loc[df_train["team"]=="georgia_lady_bulldogs",["TOVPerGame"]].head(10)

,TOVPerGame
0,0.000000
11,18.000000
18,17.500000
33,18.000000
65,17.500000
71,16.000000
81,16.000000
103,16.285714
115,16.625000
123,17.111111


In [20]:
df.loc[df["team"]=="georgia_lady_bulldogs",["TOV", "cumulative_TOV", "cumulative_games_played"]].head(10)

,TOV,cumulative_TOV,cumulative_games_played
0,18,0.0,0
11,17,18.0,1
18,19,35.0,2
33,16,54.0,3
65,10,70.0,4
71,16,80.0,5
81,18,96.0,6
103,19,114.0,7
115,21,133.0,8
123,11,154.0,9


In [21]:
df["current_win_streak"] = 0
df["longest_win_streak"] = 0

# Compute win streak features per team
for team, group in df.groupby("team"):
    streak = 0
    longest_streak = 0
    streaks = []
    longest_streaks = []

    # Shift "Won" column to ensure we only use past data
    past_wins = group["Won"].shift(1).fillna(0)  # First game has no past info, so assume 0

    for won in past_wins:
        if won == 1:
            streak += 1
        else:
            streak = 0  # Reset streak
        
        longest_streak = max(longest_streak, streak)
        
        streaks.append(streak)
        longest_streaks.append(longest_streak)

    # Assign computed streaks back to df_train
    df.loc[group.index, "current_win_streak"] = streaks
    df.loc[group.index, "longest_win_streak"] = longest_streaks

In [22]:
df_train["current_win_streak"]=df["current_win_streak"]
df_train["longest_win_streak"]=df["longest_win_streak"]

In [23]:
#Spliting Teams
df_main_team = df_train.iloc[::2].reset_index(drop=True) 
df_opp_team = df_train.iloc[1::2].reset_index(drop=True)

df_main_team = pd.DataFrame(df_main_team)
df_opp_team = pd.DataFrame(df_opp_team)

In [24]:
df_train

,game_id,game_date,team,Region,WinRatioPerGame,PtsPerGame,PointRatio,FGPerGame,FGAPerGame,FGRatio,...,eFGPerGame,OREBinPossesions,DREBinPossesions,largestLeadPerGame,OREBRatio,DREBRatio,ASTPerGame,PowerRank,current_win_streak,longest_win_streak
0,2022_2011,2021-12-30,georgia_lady_bulldogs,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
1,2022_2011,2021-12-30,lsu_tigers,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
2,2022_2012,2021-12-30,missouri_tigers,South,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
3,2022_2012,2021-12-30,south_carolina_gamecocks,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
4,2022_2013,2021-12-30,tennessee_lady_volunteers,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10433,2022_4795,2022-03-01,xavier_musketeers,East,0.310345,62.655172,0.860730,26.172414,69.655172,0.375743,...,0.393746,0.307897,0.629104,6.482759,11.965517,24.448276,12.793103,25.895827,0,3
10434,2022_4968,2022-03-04,harvard_crimson,East,0.480000,71.080000,1.048996,35.680000,94.840000,0.376213,...,0.423343,0.371459,0.589717,15.120000,14.160000,22.480000,15.240000,22.767969,0,4
10435,2022_4968,2022-03-04,dartmouth_big_green,East,0.120000,46.800000,0.728518,22.800000,75.560000,0.301747,...,0.337341,0.334911,0.708876,3.760000,11.320000,23.960000,9.240000,15.326778,0,1
10436,2022_5067,2022-03-06,harvard_crimson,East,0.500000,71.615385,1.066438,36.192308,95.346154,0.379589,...,0.427182,0.353398,0.572816,16.076923,14.000000,22.692308,15.423077,24.114123,1,4


In [25]:
df_train = df_train.rename_axis(None)  # Removes the index name

In [26]:
pd.set_option('display.max_columns', None)
df_train.describe() 

,WinRatioPerGame,PtsPerGame,PointRatio,FGPerGame,FGAPerGame,FGRatio,FG_3_PerGame,FGA_3_PerGame,FG_3_Ratio,FTMPerGame,FTAPerGame,FTRatio,STLPerGame,TOVPerGame,BLKPerGame,FoulPerGame,FG_2_PPerGame,FG_3_PPerGame,eFGPerGame,OREBinPossesions,DREBinPossesions,largestLeadPerGame,OREBRatio,DREBRatio,ASTPerGame,PowerRank,current_win_streak,longest_win_streak
count,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000
mean,0.461966,60.826877,0.954019,27.852188,73.848691,0.356542,5.537185,18.037765,0.287974,10.660387,15.152919,0.664709,7.418559,15.312767,3.016412,15.706252,0.378657,0.286497,0.392461,0.342109,0.769131,11.079940,10.944179,23.936511,12.241242,12.780660,1.397777,3.352079
std,0.278179,16.572495,0.288218,7.824424,18.961023,0.093441,2.203329,6.113402,0.084326,3.635049,4.970663,0.170308,2.597110,4.659895,1.447556,4.455020,0.098973,0.084280,0.103272,0.222950,0.544534,6.660954,3.583455,6.373803,3.907169,9.443290,2.417607,2.993575
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.260870,58.125000,0.875000,26.250000,72.410985,0.350633,4.400000,15.500000,0.269841,9.285714,13.305184,0.662083,6.300000,14.000000,2.142857,14.823529,0.373788,0.268745,0.385255,0.225970,0.510421,6.666667,9.571429,23.125000,10.880588,4.830296,0.000000,1.000000
50%,0.478261,63.711310,0.993452,29.000000,77.226496,0.374201,5.600000,18.431677,0.302673,11.000000,15.666667,0.700000,7.500000,15.740741,3.000000,16.300000,0.397949,0.301509,0.411951,0.308297,0.682406,10.555556,11.250000,25.000000,12.692308,11.490874,0.000000,3.000000
75%,0.666667,68.951786,1.102966,31.750000,81.772727,0.398523,6.833333,21.470588,0.330769,12.750000,18.027778,0.740035,8.881765,17.700000,3.900000,17.913043,0.422598,0.329109,0.439205,0.398113,0.880745,14.940257,13.000000,26.928571,14.432143,19.221275,2.000000,5.000000
max,1.000000,114.000000,2.435897,60.000000,121.000000,0.573770,19.000000,45.000000,0.727273,40.000000,46.000000,1.000000,20.000000,38.000000,13.000000,33.000000,0.602941,0.727273,0.631313,1.000000,8.333333,63.000000,27.000000,41.000000,31.000000,45.921987,28.000000,28.000000


In [27]:
#non-numerical columns
non_numeric_cols = ["game_id", "team", "Region","game_date"]

# Select only the numerical columns for subtraction
numeric_cols = [col for col in df_main_team.columns if col not in non_numeric_cols]

df_main_diff = df_main_team.copy()  # retain non-numeric columns
df_main_diff[numeric_cols] = df_main_team[numeric_cols] - df_opp_team[numeric_cols] #subtract datafram

numeric_cols = [col for col in df_opp_team.columns if col not in non_numeric_cols]

#element-wise subtraction on numerical columns
df_opp_diff = df_opp_team.copy()  # Make a copy to retain non-numeric columns
df_opp_diff[numeric_cols] = df_opp_team[numeric_cols] - df_main_team[numeric_cols]

In [28]:
df_train_final = pd.concat([df_main_diff, df_opp_diff], ignore_index=True) # concat

In [29]:
df_train_final['sort_index'] = np.arange(len(df_train_final)) % (len(df_train_final) // 2) * 2 + (np.arange(len(df_train_final)) // (len(df_train_final) // 2))

# Sort based on the new index and drop it afterward
df_train_final = df_train_final.sort_values(by='sort_index').drop(columns=['sort_index']).reset_index(drop=True)
df_train_final["Venue"] = df["home_away_NS"]
df_train_final["Results"] = df["Won"]

In [30]:
df_train_final

,game_id,game_date,team,Region,WinRatioPerGame,PtsPerGame,PointRatio,FGPerGame,FGAPerGame,FGRatio,FG_3_PerGame,FGA_3_PerGame,FG_3_Ratio,FTMPerGame,FTAPerGame,FTRatio,STLPerGame,TOVPerGame,BLKPerGame,FoulPerGame,FG_2_PPerGame,FG_3_PPerGame,eFGPerGame,OREBinPossesions,DREBinPossesions,largestLeadPerGame,OREBRatio,DREBRatio,ASTPerGame,PowerRank,current_win_streak,longest_win_streak,Venue,Results
0,2022_2011,2021-12-30,georgia_lady_bulldogs,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,0
1,2022_2011,2021-12-30,lsu_tigers,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,-1,1
2,2022_2012,2021-12-30,missouri_tigers,South,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,1
3,2022_2012,2021-12-30,south_carolina_gamecocks,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,-1,0
4,2022_2013,2021-12-30,tennessee_lady_volunteers,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10433,2022_4795,2022-03-01,xavier_musketeers,East,-0.289655,-6.478161,-0.149992,-6.960920,-12.878161,-0.025711,-4.375862,-10.356322,-0.079334,3.067816,4.582759,-0.027048,0.187356,3.686207,-0.467816,3.558621,-0.027052,-0.073476,-0.048516,0.032105,-0.167628,-4.817241,2.965517,-1.551724,-2.306897,-1.842330,-1,-3,1,0
10434,2022_4968,2022-03-04,harvard_crimson,East,0.360000,24.280000,0.320478,12.880000,19.280000,0.074466,3.640000,6.320000,0.074578,2.160000,2.640000,0.037194,4.360000,-5.520000,0.400000,1.360000,0.074783,0.076272,0.086002,0.036547,-0.119159,11.360000,2.840000,-1.480000,6.000000,7.441192,0,3,-1,1
10435,2022_4968,2022-03-04,dartmouth_big_green,East,-0.360000,-24.280000,-0.320478,-12.880000,-19.280000,-0.074466,-3.640000,-6.320000,-0.074578,-2.160000,-2.640000,-0.037194,-4.360000,5.520000,-0.400000,-1.360000,-0.074783,-0.076272,-0.086002,-0.036547,0.119159,-11.360000,-2.840000,1.480000,-6.000000,-7.441192,0,-3,1,0
10436,2022_5067,2022-03-06,harvard_crimson,East,-0.346154,2.884615,-0.275154,4.269231,15.192308,-0.018684,2.884615,9.500000,-0.017302,-2.769231,-2.230769,-0.086515,0.538462,-0.846154,0.038462,1.115385,-0.016214,-0.024803,-0.013757,-0.027504,-0.126030,-7.192308,0.038462,-2.923077,1.692308,-14.911202,-5,-8,1,0


In [31]:
home_win_rate = df_train_final[df_train_final["Venue"] == 1]["Results"].mean()
away_win_rate = df_train_final[df_train_final["Venue"] == -1]["Results"].mean()
neutral_win_rate = df_train_final[df_train_final["Venue"] == 0]["Results"].mean()

print(f"Home Win Rate: {home_win_rate:.2f}")
print(f"Away Win Rate: {away_win_rate:.2f}")
print(f"Neutral Win Rate: {neutral_win_rate:.2f}")

Home Win Rate: 0.59
Away Win Rate: 0.41
Neutral Win Rate: 0.50


In [32]:
#Linear realtionships with Results
nonnumeric = df_train_final[["team","game_id","Region","game_date"]]
#pd.set_option("display.max_rows", None) #Displays full column
df_train_final=df_train_final.drop(columns=nonnumeric)
#monotonic realtionships with Results
spearman = df_train_final.corr(method="spearman").iloc[-1]
spearman.sort_values(ascending = False)

Results               1.000000
PointRatio            0.479224
WinRatioPerGame       0.463672
largestLeadPerGame    0.448175
longest_win_streak    0.421080
PtsPerGame            0.394640
FG_2_PPerGame         0.383842
FGRatio               0.376291
FGPerGame             0.370458
eFGPerGame            0.367235
ASTPerGame            0.358451
PowerRank             0.354167
current_win_streak    0.321970
FG_3_Ratio            0.262385
FG_3_PPerGame         0.251436
DREBRatio             0.242697
OREBinPossesions      0.229423
BLKPerGame            0.223965
DREBinPossesions      0.218891
FTMPerGame            0.212651
FGAPerGame            0.203571
FG_3_PerGame          0.202235
FTRatio               0.185846
FTAPerGame            0.178239
Venue                 0.175287
STLPerGame            0.164698
OREBRatio             0.156574
FGA_3_PerGame         0.129010
FoulPerGame          -0.041563
TOVPerGame           -0.163203
Name: Results, dtype: float64

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import StandardScaler

# Ensure no target leakage (drop problematic columns)
x = df_train_final.drop(columns=["Results"], errors='ignore')
y = df_train_final["Results"].values

# Train-test split BEFORE transformations
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)
print("Before Scaling - Train Min:", x_train.min().min(), "Max:", x_train.max().max())
print("Before Scaling - Test Min:", x_test.min().min(), "Max:", x_test.max().max())

# Min-max scaling (only using x_train values)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)  # Fit on train
x_test = scaler.transform(x_test)  # Transform test using train stats

Before Scaling - Train Min: -107.0 Max: 116.0
Before Scaling - Test Min: -116.0 Max: 97.0


In [34]:
print("After Scaling - Train Min:", x_train.min().min(), "Max:", x_train.max().max())
print("After Scaling - Test Min:", x_test.min().min(), "Max:", x_test.max().max())

After Scaling - Train Min: -7.2239336621667665 Max: 8.366934701089008
After Scaling - Test Min: -8.377560882385021 Max: 7.243136459833694


In [35]:
poly_svc = SVC(kernel='rbf', C=0.5, gamma=0.05994842503189409)
poly_svc.fit(x_train, y_train)

y_train_pred = poly_svc.predict(x_train)
y_pred = poly_svc.predict(x_test)
#Predict on test set
print('Training Accuracy:', accuracy_score(y_train, y_train_pred))
print('Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

#poly_svc = SVC(kernel='rbf', C=1, gamma="auto")
#Training Accuracy: 0.7505389221556886
#Model accuracy score: 0.7399

Training Accuracy: 0.7518562874251496
Model accuracy score: 0.7375


In [36]:
df_train.columns

Index(['game_id', 'game_date', 'team', 'Region', 'WinRatioPerGame',
       'PtsPerGame', 'PointRatio', 'FGPerGame', 'FGAPerGame', 'FGRatio',
       'FG_3_PerGame', 'FGA_3_PerGame', 'FG_3_Ratio', 'FTMPerGame',
       'FTAPerGame', 'FTRatio', 'STLPerGame', 'TOVPerGame', 'BLKPerGame',
       'FoulPerGame', 'FG_2_PPerGame', 'FG_3_PPerGame', 'eFGPerGame',
       'OREBinPossesions', 'DREBinPossesions', 'largestLeadPerGame',
       'OREBRatio', 'DREBRatio', 'ASTPerGame', 'PowerRank',
       'current_win_streak', 'longest_win_streak'],
      dtype='object')

In [37]:
df_sorted = df_train.sort_values(by='game_date')  # Replace 'date' with the actual date column name

# Group by 'team' and pick the last row for each team based on the sorted date
last_game_stats = df_sorted.groupby('team').last().reset_index()

# Now, df_last_game contains the stats for each team's last game.
# If you want to merge this with your df_train, you can do that like so:
last_game_stats = last_game_stats[['game_id', 'game_date', 'team', 'Region', 'WinRatioPerGame',
       'PtsPerGame', 'PointRatio', 'FGPerGame', 'FGAPerGame', 'FGRatio',
       'FG_3_PerGame', 'FGA_3_PerGame', 'FG_3_Ratio', 'FTMPerGame',
       'FTAPerGame', 'FTRatio', 'STLPerGame', 'TOVPerGame', 'BLKPerGame',
       'FoulPerGame', 'FG_2_PPerGame', 'FG_3_PPerGame', 'eFGPerGame',
       'OREBinPossesions', 'DREBinPossesions', 'largestLeadPerGame',
       'OREBRatio', 'DREBRatio', 'ASTPerGame', 'PowerRank',
       'current_win_streak', 'longest_win_streak']]
last_game_stats = pd.DataFrame(last_game_stats)
#last_game_stats.drop(["game_id", "game_date"], axis=1, inplace=True)

In [38]:
East_Teams = last_game_stats[last_game_stats["Region"] == "East"]
West_Teams = last_game_stats[last_game_stats["Region"] == "West"]
North_Teams = last_game_stats[last_game_stats["Region"] == "North"]
South_Teams = last_game_stats[last_game_stats["Region"] == "South"]

Tourney_West = West_Teams.sort_values(by="PowerRank", ascending=False).head(16)
Tourney_East = East_Teams.sort_values(by="PowerRank", ascending=False).head(16)
Tourney_North = North_Teams.sort_values(by="PowerRank", ascending=False).head(16)
Tourney_South = South_Teams.sort_values(by="PowerRank", ascending=False).head(16)

In [39]:
#adding seeds
for df in [Tourney_East, Tourney_West, Tourney_North, Tourney_South]:
    df.insert(2, "Seed", range(1, len(df) + 1))

#Currently unsure about the seeds since ML thinks 16 is greater than 1

In [40]:
Tourney_West_HomeTeam = Tourney_West.head(8)
Tourney_West_AwayTeam = Tourney_West.tail(8).sort_values(by="Seed",ascending=False)

In [41]:
Tourney_West_HomeTeam = Tourney_West_HomeTeam.reset_index(drop=True)
Tourney_West_AwayTeam = Tourney_West_AwayTeam.reset_index(drop=True)

non_numeric_cols = ["game_id", "team", "Region","game_date","Seed"]

numeric_cols = [col for col in Tourney_West_HomeTeam.columns if col not in non_numeric_cols]

Tourney_West_HomeTeam_diff = Tourney_West_HomeTeam.copy()  # retain non-numeric columns
Tourney_West_HomeTeam_diff[numeric_cols] = Tourney_West_HomeTeam[numeric_cols] - Tourney_West_AwayTeam[numeric_cols] #subtract datafram


numeric_cols = [col for col in Tourney_West_AwayTeam.columns if col not in non_numeric_cols]
#element-wise subtraction on numerical columns
Tourney_West_AwayTeam_diff = Tourney_West_AwayTeam.copy()  # Make a copy to retain non-numeric columns
Tourney_West_AwayTeam_diff[numeric_cols] = Tourney_West_AwayTeam[numeric_cols] - Tourney_West_HomeTeam[numeric_cols]

In [42]:
Tourney_West_FirstRound = pd.concat([Tourney_West_HomeTeam_diff, Tourney_West_AwayTeam_diff], ignore_index=True)

In [43]:
nonnumeric = Tourney_West_FirstRound[["game_id","Region","game_date"]]
#pd.set_option("display.max_rows", None) #Displays full column
Tourney_West_FirstRound=Tourney_West_FirstRound.drop(columns=nonnumeric)
Tourney_West_FirstRound["Venue"] = 0

In [51]:
# Initialize SVC with probability=True to enable predict_proba
poly_svc = SVC(kernel='rbf', C=0.5, gamma=0.05994842503189409, probability=True)

# Train the model
poly_svc.fit(x_train, y_train)

# Step 1: Ensure only trained feature columns are used
feature_cols = df_train_final.drop(columns=["Results"]).columns  # Get original feature names

# Step 2: Select the relevant features from the tournament dataset
X_tournament = Tourney_West_FirstRound[feature_cols]

# Step 3: Apply the same StandardScaler transformation as in training
X_tournament_scaled = scaler.transform(X_tournament)  # Use the trained scaler

# Step 4: Predict winners and probabilities
predictions = poly_svc.predict(X_tournament_scaled)
probabilities = poly_svc.predict_proba(X_tournament_scaled)  # Probabilities for both classes

# Step 5: Add predictions and probabilities to the DataFrame
Tourney_West_FirstRound["Predicted_Winner"] = predictions
Tourney_West_FirstRound["Winner_Confidence"] = probabilities[:, 1]  # Probability of the winning class (class 1)

# Step 6: Display results with predictions and confidence
print(Tourney_West_FirstRound[["team", "Predicted_Winner", "Winner_Confidence", "Seed"]])  # Adjust as needed

                          team  Predicted_Winner  Winner_Confidence  Seed
0             gonzaga_bulldogs                 1           0.625183     1
1           colorado_buffaloes                 1           0.787868     2
2             unlv_lady_rebels                 0           0.399578     3
3             oklahoma_sooners                 0           0.195630     4
4            stanford_cardinal                 1           0.781401     5
5             new_mexico_lobos                 0           0.211361     6
6                    utah_utes                 0           0.451564     7
7   california_baptist_lancers                 0           0.249379     8
8       ut_arlington_mavericks                 0           0.307130    16
9       long_beach_state_beach                 0           0.223040    15
10        nebraska_cornhuskers                 1           0.562513    14
11                 byu_cougars                 1           0.827872    13
12    washington_state_cougars        

In [52]:
# Select only the rows where Predicted_Winner is 1 (winning teams)
Tourney_West_SecondRound = Tourney_West_FirstRound[Tourney_West_FirstRound["Predicted_Winner"] == 1].copy()

# Get the list of winning teams (teams with Predicted_Winner = 1)
winning_teams = Tourney_West_SecondRound[Tourney_West_SecondRound["Predicted_Winner"] == 1]["team"]
winning_teams

0         gonzaga_bulldogs
1       colorado_buffaloes
4        stanford_cardinal
10    nebraska_cornhuskers
11             byu_cougars
13            baylor_bears
14        arizona_wildcats
15         texas_longhorns
Name: team, dtype: object

In [53]:
#Grabbing winner in make Tourney_West_SecondRound
# Get the original features (before subtraction) of these winning teams from Tourney_West
Tourney_West_SecondRound = Tourney_West[Tourney_West["team"].isin(winning_teams)]
# Now, Tourney_West_original_features contains the original, unsubtracted features of the winning teams
Tourney_West_SecondRound = Tourney_West_SecondRound.sort_values(by="Seed")

In [54]:
#Splitting Tourney_West_SecondRound
# Step 2: List of seed pairs for the second round
seed_pairs = [(1, 16), (5, 12), (3, 14), (7, 10)]
# Step 3: Initialize an empty list to store the main teams for the second round
main_teams = []

# Step 4: Loop over each pair of seeds
for seed_pair in seed_pairs:
    # Check for the winner from the first round for each seed
    winner = Tourney_West_SecondRound[Tourney_West_SecondRound['Seed'].isin(seed_pair)]
    
    # Add the winning team to the main_teams list
    if not winner.empty:
        main_teams.append(winner)

# Step 5: Concatenate the winners into the main team dataframe
West2ndHomeTeam = pd.concat(main_teams, ignore_index=True)

In [55]:
#Splitting Tourney_West_SecondRound
# Step 2: List of seed pairs for the second round (opposite order for opponents)
seed_pairs2 = [(8,9), (4, 13), (6, 11), (2, 15)]

# Step 3: Initialize an empty list to store the main teams for the second round
opponent_teams = []

# Step 4: Loop over each pair of seeds
for seed_pair2 in seed_pairs2:
    # Check for the winner from the first round for each seed
    winner = Tourney_West_SecondRound[Tourney_West_SecondRound['Seed'].isin(seed_pair2)]
    
    # Add the winning team to the main_teams list
    if not winner.empty:
        opponent_teams.append(winner)

# Step 5: Concatenate the winners into the main team dataframe
West2ndAwayTeam = pd.concat(opponent_teams, ignore_index=True)

# Now you have the main teams for the second round

In [56]:
# Reset index for both dataframes
West2ndHomeTeam = West2ndHomeTeam.reset_index(drop=True)
West2ndAwayTeam = West2ndAwayTeam.reset_index(drop=True)

# List of non-numeric columns
non_numeric_cols = ["game_id", "team", "Region", "game_date", "Seed"]

# Identify numeric columns for the Main Team (Home Team)
numeric_cols = [col for col in West2ndHomeTeam.columns if col not in non_numeric_cols]

# Subtract the features for Home Team and Away Team
West2ndHomeTeam_diff = West2ndHomeTeam.copy()  # retain non-numeric columns
West2ndHomeTeam_diff[numeric_cols] = West2ndHomeTeam[numeric_cols] - West2ndAwayTeam[numeric_cols]  # subtraction

# Identify numeric columns for the Away Team
numeric_cols = [col for col in West2ndAwayTeam.columns if col not in non_numeric_cols]

# Subtract the features for Away Team (check subtraction with correct dataframe)
West2ndAwayTeam_diff = West2ndAwayTeam.copy()  # retain non-numeric columns
West2ndAwayTeam_diff[numeric_cols] = West2ndAwayTeam[numeric_cols] - West2ndHomeTeam[numeric_cols]  # correct subtraction with Home Team

# Combine the dataframes into a final one
West2nd_Final = pd.concat([West2ndHomeTeam_diff, West2ndAwayTeam_diff], ignore_index=True)

In [58]:
West2nd_Final["Venue"] = 0
# Step 1: Ensure only trained feature columns are used
feature_cols = df_train_final.drop(columns=["Results"]).columns  # Get original feature names

# Step 2: Select the relevant features from the tournament dataset
X_tournament = West2nd_Final[feature_cols]

# Step 3: Apply the same StandardScaler transformation as in training
X_tournament_scaled = scaler.transform(X_tournament)  # Use the trained scaler

# Step 4: Predict winners and probabilities
predictions = poly_svc.predict(X_tournament_scaled)
probabilities = poly_svc.predict_proba(X_tournament_scaled)  # Probabilities for both classes

# Step 5: Add predictions and probabilities to the DataFrame
West2nd_Final["Predicted_Winner"] = predictions
West2nd_Final["Winner_Confidence"] = probabilities[:, 1]  # Probability of the winning class (class 1)

# Step 6: Display results with predictions and confidence
West2nd_Final[["team", "Predicted_Winner", "Winner_Confidence", "Seed"]]  # Adjust as needed

,team,Predicted_Winner,Winner_Confidence,Seed
0,gonzaga_bulldogs,0,0.347554,1
1,stanford_cardinal,1,0.634081,5
2,nebraska_cornhuskers,0,0.328402,14
3,arizona_wildcats,0,0.358224,10
4,texas_longhorns,1,0.629075,9
5,byu_cougars,0,0.353063,13
6,baylor_bears,1,0.640672,11
7,colorado_buffaloes,1,0.623617,2


In [59]:
# Select only the rows where Predicted_Winner is 1
Tourney_West_Sweet16 = West2nd_Final[West2nd_Final["Predicted_Winner"] == 1].copy()
# Get the list of winning teams
winning_teams = Tourney_West_Sweet16[Tourney_West_Sweet16["Predicted_Winner"] == 1]["team"]
winning_teams

1     stanford_cardinal
4       texas_longhorns
6          baylor_bears
7    colorado_buffaloes
Name: team, dtype: object

In [60]:
West_Sweet16 = Tourney_West[Tourney_West["team"].isin(winning_teams)]

In [61]:
West_Sweet16

,game_id,game_date,Seed,team,Region,WinRatioPerGame,PtsPerGame,PointRatio,FGPerGame,FGAPerGame,FGRatio,FG_3_PerGame,FGA_3_PerGame,FG_3_Ratio,FTMPerGame,FTAPerGame,FTRatio,STLPerGame,TOVPerGame,BLKPerGame,FoulPerGame,FG_2_PPerGame,FG_3_PPerGame,eFGPerGame,OREBinPossesions,DREBinPossesions,largestLeadPerGame,OREBRatio,DREBRatio,ASTPerGame,PowerRank,current_win_streak,longest_win_streak
99,2022_4954,2022-03-04,2,colorado_buffaloes,West,0.785714,66.071429,1.182109,29.928571,76.750000,0.389949,5.892857,19.107143,0.308411,12.107143,17.892857,0.676647,10.821429,14.178571,3.714286,17.464286,0.415527,0.307470,0.428352,0.302893,0.619349,16.571429,11.964286,24.464286,14.500000,41.295109,6,13
441,2022_5057,2022-03-06,5,stanford_cardinal,West,0.892857,73.785714,1.286426,34.821429,80.892857,0.430464,7.178571,19.964286,0.359571,11.321429,16.857143,0.671610,8.607143,13.785714,5.892857,16.107143,0.454822,0.362201,0.476801,0.345249,0.746002,21.214286,13.107143,28.321429,15.142857,39.131176,22,22
464,2022_5214,2022-03-13,9,texas_longhorns,West,0.827586,72.448276,1.295314,31.655172,76.000000,0.416515,4.862069,13.965517,0.348148,14.000000,19.965517,0.701209,9.965517,14.103448,3.758621,18.344828,0.431854,0.333229,0.448401,0.324624,0.528108,21.586207,14.137931,23.000000,12.724138,36.503303,9,9
31,2022_5214,2022-03-13,11,baylor_bears,West,0.833333,77.600000,1.285478,35.066667,80.733333,0.434352,6.733333,19.000000,0.354386,14.200000,19.466667,0.729452,6.700000,12.533333,4.100000,14.433333,0.459472,0.349328,0.475156,0.341560,0.842053,23.600000,11.533333,28.433333,17.600000,36.066057,6,12


In [65]:
# Step 1: List of seed pairs for the Sweet 16
seed_pairs_sweet16 = [(1, 8), (5, 4), (3, 6), (7, 2)]

# Step 2: Initialize an empty list to store the main teams for the Sweet 16
sweet16_teams = []

# Step 3: Loop over each pair of seeds
for seed_pair in seed_pairs_sweet16:
    # Check for the winner from the second round for each seed
    winner = West_Sweet16[West_Sweet16['Seed'].isin(seed_pair)]
    
    # Add the winning team to the sweet16_teams list
    if not winner.empty:
        sweet16_teams.append(winner)

# Step 4: Concatenate the winners into the Sweet 16 team dataframe
West2ndHomeTeam_Sweet16 = pd.concat(sweet16_teams, ignore_index=True)

In [66]:
West2ndHomeTeam_Sweet16

,game_id,game_date,Seed,team,Region,WinRatioPerGame,PtsPerGame,PointRatio,FGPerGame,FGAPerGame,FGRatio,FG_3_PerGame,FGA_3_PerGame,FG_3_Ratio,FTMPerGame,FTAPerGame,FTRatio,STLPerGame,TOVPerGame,BLKPerGame,FoulPerGame,FG_2_PPerGame,FG_3_PPerGame,eFGPerGame,OREBinPossesions,DREBinPossesions,largestLeadPerGame,OREBRatio,DREBRatio,ASTPerGame,PowerRank,current_win_streak,longest_win_streak
0,2022_5057,2022-03-06,5,stanford_cardinal,West,0.892857,73.785714,1.286426,34.821429,80.892857,0.430464,7.178571,19.964286,0.359571,11.321429,16.857143,0.671610,8.607143,13.785714,5.892857,16.107143,0.454822,0.362201,0.476801,0.345249,0.746002,21.214286,13.107143,28.321429,15.142857,39.131176,22,22
1,2022_4954,2022-03-04,2,colorado_buffaloes,West,0.785714,66.071429,1.182109,29.928571,76.750000,0.389949,5.892857,19.107143,0.308411,12.107143,17.892857,0.676647,10.821429,14.178571,3.714286,17.464286,0.415527,0.307470,0.428352,0.302893,0.619349,16.571429,11.964286,24.464286,14.500000,41.295109,6,13
